In [4]:
search_for = 124
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0038514137268066406
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 621252555
type: [1, 1, 1, 1, 124] 124
cases of this type: 236421376
100000 25.363240700219706
200000 40.49921593240223
300000 44.65953377700827
400000 44.2097362827231
500000 45.37876676497792
600000 44.27019619096217
700000 43.474926168556834
800000 43.80483857865635
900000 43.37742867016952
1000000 15.206781833009824
1100000 44.78969016702938
1200000 43.29727804670034
1300000 43.06419956125305
1400000 43.26031219322677
1500000 45.17210032740826
1600000 43.61304417552556
1700000 43.23084849151971
1800000 43.92151746299888
1900000 38.001922849163115
2000000 21.786997715930603
2100000 43.059387687604044
2200000 42.871707178527025
2300000 41.91258036408515
2400000 43.62440065777311
2500000 42.692787358784194
2600000 41.77120164715506
2700000 42.1499560127372
2800000 42.68286695793284
2900000 17.904158237642715
3000000 40.29213932

29800000 41.320479174262225
29900000 42.34508088305828
30000000 42.910240869832776
30100000 43.23451241725749
30200000 44.78883975142274
30300000 26.845122967597483
30400000 39.905479491908025
30500000 45.91419528227868
30600000 44.06491098798722
30700000 44.141278245902406
30800000 20.68616207471394
30900000 45.51697925743478
31000000 44.57093510720879
31100000 43.04956146750762
31200000 42.958244428269026
31300000 39.57981145071857
31400000 39.528892515873665
31500000 17.192899214187246
31600000 40.5824675928108
31700000 39.33914038361526
31800000 41.10856564716724
31900000 41.67401692152545
32000000 44.05139470781642
32100000 43.90087216074594
32200000 24.55713399146015
32300000 40.47398640123469
32400000 43.47100665240502
32500000 45.57984849477765
32600000 45.102171607683644
32700000 27.242941839349054
32800000 39.26463064527816
32900000 46.15653592888586
33000000 43.531009869273845
33100000 43.25705326762245
33200000 41.26937965645251
33300000 39.66608946951054
33400000 19.388087

59800000 38.582119174090835
59900000 38.171461100142906
60000000 38.51026755535677
60100000 15.19482684463489
60200000 39.118884914722834
60300000 37.56491813009987
60400000 38.771214964405985
60500000 38.46875551170383
60600000 19.91570947216904
60700000 41.02009997555097
60800000 41.16494123384453
60900000 41.29986528719231
61000000 44.14603319204281
61100000 41.868211385274186
61200000 41.443920389484276
61300000 40.207813917400415
61400000 39.5525037064277
61500000 31.834674531464664
61600000 23.87868787093597
61700000 35.16365820863441
61800000 36.73569581985864
61900000 34.583737945117264
62000000 17.608403324226
62100000 33.754429371766875
62200000 36.199735446336035
62300000 34.09342063104869
62400000 35.495305852997504
62500000 20.574399798414223
62600000 38.73975923737161
62700000 38.21289514356318
62800000 39.55190324968437
62900000 40.37976483418973
63000000 41.92882342579439
63100000 41.80995088217004
63200000 41.475344986255074
63300000 39.972733931078544
63400000 38.6276

89800000 40.30015282339661
89900000 37.73195381255825
90000000 36.1016282887129
90100000 36.28427339149501
90200000 36.36068725707985
90300000 36.0321356199347
90400000 16.964557317606925
90500000 33.97035860329071
90600000 16.58587956080377
90700000 33.64149738768747
90800000 27.806002316559383
90900000 27.29315489194443
91000000 36.13622934556335
91100000 34.80395777185434
91200000 36.45487848526606
91300000 36.80937570291292
91400000 38.83962645721996
91500000 38.890881062502714
91600000 39.10302282404175
91700000 38.40916019681042
91800000 36.43385620037767
91900000 34.08417824281739
92000000 34.847133993346624
92100000 35.35353339747972
92200000 34.37252343476615
92300000 16.287490904417258
92400000 34.58839402139437
92500000 17.537227011438155
92600000 31.099737178860558
92700000 23.71385565700646
92800000 28.62049947179539
92900000 34.580777302759635
93000000 35.39750227458039
93100000 33.89163005634661
93200000 34.65488765098322
93300000 37.53298954831019
93400000 37.9943676648

119100000 3.6474789465328064
119200000 3.658902988994363
119300000 3.678247988210811
119400000 3.741410481675476
119500000 3.747265388777508
119600000 3.653797100053029
119700000 3.634111087340236
119800000 3.664130903190282
119900000 3.6467235265383655
120000000 3.627924891839355
120100000 3.6324053154768507
120200000 3.638081582945748
120300000 3.6108562082169655
120400000 3.6355255045759707
120500000 3.6627505246008205
120600000 3.6882233118050096
120700000 3.627638760101792
120800000 3.627618004656378
120900000 3.6099614350097275
121000000 3.6225282306426303
121100000 3.61684446496236
121200000 3.6201238581317967
121300000 3.610659055522614
121400000 3.615982625305166
121500000 3.6362614406210483
121600000 3.695118894198189
121700000 3.8379678093566363
121800000 3.6237885440229376
121900000 3.6424861417784355
122000000 3.6175609868872955
122100000 3.6496966052989066
122200000 3.603388834934496
122300000 3.6153799309731256
122400000 3.614043478852014
122500000 3.5992041679541207
122

...

493100000 4.650885615118503
493200000 4.411361399519672
493300000 4.084949731339534
493400000 1.4939973922374679
493500000 3.7320431738601627
493600000 3.2972241521512635
493700000 3.0313716672626696
493800000 3.7342091423722206
493900000 4.068304117713279
494000000 4.153398605851527
494100000 4.869371193827937
494200000 3.983641676365306
494300000 2.599042747804516
494400000 2.6411304864585894
494500000 3.5486493542346857
494600000 2.5399560929980876
494700000 3.7862739883864527
494800000 4.12315962302612
494900000 4.255159238555925
495000000 4.05731789446785
495100000 4.520969961489296
495200000 4.188584493508339
495300000 1.261047007713298
495400000 3.6066054192061756
495500000 2.2813028519892757
495600000 3.8333198124409917
495700000 3.9893143239648907
495800000 3.912348073189752
495900000 3.8651621039744217
496000000 3.9442504163506293
496100000 4.41079044249587
496200000 2.6292691442660487
496300000 2.5378333851845762
496400000 2.690160745361547
496500000 3.469316427188426
496600

521100000 3.2194450207547844
521200000 3.1484781866885787
521300000 3.049583868700107
521400000 3.210622398171028
521500000 3.224247151255436
521600000 3.202105893573218
521700000 3.0463221693496405
521800000 3.1572052720530466
521900000 3.3356550161281757
522000000 3.2656188393626215
522100000 3.1543018480748173
522200000 3.2783035372967326
522300000 3.225225890250862
522400000 3.3294138150781727
522500000 3.230639809486074
522600000 3.2061359860559104
522700000 3.280816327501284
522800000 3.3400325465032754
522900000 3.1808011022055642
523000000 3.252190839023646
523100000 3.145646213349193
523200000 3.3210380551219885
523300000 3.2119984884789288
523400000 3.258486494403319
523500000 3.0109442475736143
523600000 2.909394411704858
523700000 2.9827414617141748
523800000 3.1242153839582203
523900000 3.1441497526281674
524000000 3.147715768903553
524100000 2.970046040720413
524200000 3.1747930723422764
524300000 2.9748741152371863
524400000 3.6920302590226526
524500000 3.398007507445368

547900000 3.651708012292574
548000000 2.0907646291852826
548100000 3.267428936182519
548200000 3.7314256483148003
548300000 1.1433835791859364
548400000 0.5504088200814823
548500000 0.5535250794988407
548600000 0.5578206973391274
548700000 0.5517573041574955
548800000 0.5370619426376091
548900000 0.5343749564771123
549000000 0.5288545896006523
549100000 0.5306104064168832
549200000 0.5303808220296071
549300000 0.5326797415752212
549400000 0.5445123734144204
549500000 0.5350898343218896
549600000 0.5211201929928064
549700000 0.5227602842945986
549800000 0.5206357487123775
549900000 0.5228580319314003
550000000 0.5206912333570752
550100000 0.5194035714370906
550200000 0.517657735326618
550300000 0.516936462663359
550400000 0.5159810197424193
550500000 0.5212541500048041
550600000 0.5351781305224266
550700000 4.342531205580688
550800000 4.081479534526716
550900000 1.5039143060319493
551000000 4.149743310615714
551100000 4.288038475417842
551200000 3.2566476887626084
551300000 2.6415836950

576300000 1.4619085985460083
576400000 1.3536440769045917
576500000 1.1222922726656563
576600000 0.7825480345883965
576700000 1.2838691772012973
576800000 1.5360271292404435
576900000 1.1020085259413819
577000000 1.2876630784569016
577100000 0.6051072843873435
577200000 1.4931258380850454
577300000 1.4564547789634898
577400000 1.0694848257234262
577500000 1.234018706253737
577600000 0.7182424564525187
577700000 1.4672867711107531
577800000 1.217273728548189
577900000 1.1555363605527977
578000000 1.0754396949024598
578100000 0.9412097727927764
578200000 1.470411056411485
578300000 0.9886134915392301
578400000 1.345649253844112
578500000 0.8117857134795752
578600000 1.2454297579927047
578700000 1.444351036230117
best so far: 0
type: [1, 2, 1, 31, 2] 124
cases of this type: 953312
578800000 0.4139686405578487
578900000 0.2629898157764773
579000000 0.2605018800764282
579100000 0.24217450939595364
579200000 0.23712299772324497
579300000 0.2338283387174904
579400000 0.23287369498397903
57950

599700000 0.15422382895270983
599800000 0.4108780634190308
599900000 0.4132447713792059
600000000 0.5033769627119005
600100000 0.4708115710398621
600200000 0.32391204463070633
600300000 0.682876970041444
600400000 0.28427709981117644
600500000 0.5934127786302533
600600000 0.6420641756521364
600700000 0.5831293201612896
600800000 0.7572299096156921
600900000 0.48062900963693855
601000000 0.6954713334528903
601100000 0.5171063447828326
601200000 0.6306565879967511
601300000 0.6351472185829415
601400000 0.5526534472009937
601500000 0.6810487527528007
601600000 0.4165137070579761
601700000 0.5694555462096565
601800000 0.586198269729048
601900000 0.6255150155306889
602000000 0.5980453615352643
602100000 0.46204545457401863
602200000 0.7116992921498617
602300000 0.48926226840780507
602400000 0.6197111996208231
602500000 0.5305508310762544
602600000 0.47790131662224733
602700000 0.6979730697986484
602800000 0.5110574087651472
602900000 0.5475927139099803
603000000 0.3716680443921288
603100000